In [ ]:
import pandas as pd
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Embedding, Input, Bidirectional
from keras.layers import GlobalMaxPooling1D, SpatialDropout1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, precision_score, recall_score
from keras.utils import plot_model

def metrics(predictions, y_test):
    tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
    print(f'Verdaderos Negativos: {tn}')
    print(f'Falsos Negativos: {fn}')
    print(f'Verdaderos Positivos: {tp}')
    print(f'Falsos Positivos: {fp}')
    print()
    print(f'precision score: {precision_score(y_test, predictions)}')
    print(f'recall score: {recall_score(y_test, predictions)}')
    print(f'f1 score: {f1_score(y_test,  predictions)}')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
!pip install gensim
from gensim.models import KeyedVectors

In [ ]:
!wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"
!wget -c http://nlp.stanford.edu/data/glove.twitter.27B.zip
!unzip glove.twitter.27B.zip

EMBEDDING_FILE = 'GoogleNews-vectors-negative300.bin.gz'
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

--2020-08-06 17:44:20--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.241.54
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.241.54|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

--2020-08-06 17:44:22--  http://nlp.stanford.edu/data/glove.twitter.27B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.twitter.27B.zip [following]
--2020-08-06 17:44:22--  https://nlp.stanford.edu/data/glove.twitter.27B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
url_train = 'https://raw.githubusercontent.com/fsicardir/datos-tp2/master/dataset/train.csv?token=AFVAIUW66UE3NA5X2SYXNPC7GHGJY'
url_test = 'https://raw.githubusercontent.com/fsicardir/datos-tp2/master/dataset/test.csv?token=AFVAIUUSBVEOOMDIFV4GU6C7GHGNK'

read_train = pd.read_csv(url_train)
read_test = pd.read_csv(url_test)

In [ ]:
df_train = read_train[['id', 'text', 'target']]
df_test = read_test[['id', 'text']]

# Limpiamos los datos de la forma usual
# Quitamos las urls
df_train['text'] = df_train['text'].str.replace(r'http:\/\/.*', '', regex=True).replace(r'https:\/\/.*', '', regex=True)
df_test['text'] = df_test['text'].str.replace(r'http:\/\/.*', '', regex=True).replace(r'https:\/\/.*', '', regex=True)

# Quitamos user mentions, signos de puntuación, hashtags y stopwords.
def clean_text(text):
    words = text.lower().split(' ')
    text = ' '.join([word for word in words if not word.startswith('@') and word not in stopwords.words('english')])
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

df_train['text'] = df_train['text'].apply(clean_text)
df_test['text'] = df_test['text'].apply(clean_text)

train_tweets = df_train['text'].tolist()
train_target = df_train['target']
test_tweets = df_test['text'].tolist()
len(train_tweets), len(train_target), len(test_tweets)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

(7613, 7613, 3263)

In [ ]:
oov_token = "<UNK>"

tokenizer = Tokenizer(oov_token=oov_token)
tokenizer.fit_on_texts(train_tweets)
vocabulary_size = len(tokenizer.word_index) + 1

sequences_train = tokenizer.texts_to_sequences(train_tweets)
sequence_test = tokenizer.texts_to_sequences(test_tweets)

max_padding = 25
padded_vecs_train = pad_sequences(sequences_train, maxlen=max_padding, padding='post')
padded_vecs_test = pad_sequences(sequence_test, maxlen=max_padding, padding='post')

# Ahora vamos a crear una matriz que tendrá los embeddings de Google
# correspondientes a cada palabra de nuestro vocabulario.
# Esto se lo pasaremos como pesos a la capa de Embedding del modelo a entrenar.
embedding_dim_w2v = 300
embedding_matrix_w2v = np.zeros((vocabulary_size, embedding_dim_w2v))
oov_words_w2v = 0
for word, i in tokenizer.word_index.items():
  try:
    embedding_vector = word2vec[word]
    embedding_matrix_w2v[i] = embedding_vector
  except:
    oov_words_w2v += 1
    continue


In [ ]:
# Cargamos los embeddings de GloVe. Utilizaremos los de 200 dimensiones.
glove_dict = {}
with open('glove.twitter.27B.200d.txt') as f:
  for line in f:
      values = line.split()
      word = values[0]
      vector = np.asarray(values[1:], dtype='float32')
      glove_dict[word] = vector

oov_words_gl = 0
embedding_dim_gl = 200
embedding_matrix_gl = np.zeros((vocabulary_size, embedding_dim_gl))
for word, i in tokenizer.word_index.items():
    embedding_vector = glove_dict.get(word, None)
    if embedding_vector is not None:
        embedding_matrix_gl[i] = embedding_vector
    else:
      oov_words_gl += 1

In [ ]:
vocabulary_size, oov_words_w2v, oov_words_gl

(15404, 4608, 3406)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(padded_vecs_train, train_target, test_size=0.2, random_state=31)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((6090, 25), (6090,), (1523, 25), (1523,))

Nuestro baseline va a ser un modelo simple, donde tenemos la siguiente arquitectura:

* Capa de Embedding: vamos a probar tanto con w2v como con glove.
* Capa LSTM: Dejamos fijo el unit_size en 64 porque nos dio resultados decentes en pruebas preliminares.
* Capa Dense: La capa FC para interpretar la salida de la RNN de la capa previa, por default usaremos 32 neuronas ya que es un número recurrente en distintos tutoriales.
* Capa Dropout: Para reducir un poco el overfit vamos a dejar una capa de Dropout configurable por parámetro. Por default será 0.
* Capa Dense: Para hacer la predicción, una neurona, función de activación sigmoidea.

In [ ]:
emb_layer_w2v =  Embedding(input_dim=vocabulary_size, output_dim=embedding_dim_w2v, weights=[embedding_matrix_w2v], input_length=max_padding, trainable=False)
emb_layer_gl = Embedding(input_dim=vocabulary_size, output_dim=embedding_dim_gl, weights=[embedding_matrix_gl], input_length=max_padding, trainable=False)

In [ ]:
# Como capa de Embedding default usaremos la de glove.
def build_baseline(lstm_size=64, hidden_neurons=32, dropout_rate=0.0, emb_layer=emb_layer_gl):
  model = Sequential()
  model.add(emb_layer)
  model.add(LSTM(lstm_size))
  model.add(Dense(hidden_neurons, activation='relu'))
  model.add(Dropout(dropout_rate))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

# Baseline + GloVe Twitter
Primero vamos a usar los parámetros default y ver qué f1 score obtenemos.

In [ ]:
epochs = 5

In [ ]:
model1 = build_baseline()
model1.summary()
model1.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test))
preds = model1.predict_classes(X_test)
metrics(preds, y_test)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 25, 200)           3080800   
_________________________________________________________________
lstm (LSTM)                  (None, 64)                67840     
_________________________________________________________________
dense (Dense)                (None, 32)                2080      
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 3,150,753
Trainable params: 69,953
Non-trainable params: 3,080,800
_________________________________________________________________
Epoch 1/5
191/191 [==============================] - 5s 24ms/step - loss: 0.5056 - accuracy: 0.7670 - val_l

Obtuvimos un puntaje decente a comparación de lo que observamos en los notebooks donde usamos CNNs sencillas, veamos a continuación cuánto cambia el resultado utilizando w2v.

# Baseline + w2v

In [ ]:
model2 = build_baseline(emb_layer=emb_layer_w2v)
model2.summary()
model2.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test))
preds = model2.predict_classes(X_test)
metrics(preds, y_test)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 25, 300)           4621200   
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                93440     
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 4,716,753
Trainable params: 95,553
Non-trainable params: 4,621,200
_________________________________________________________________
Epoch 1/5
191/191 [==============================] - 5s 29ms/step - loss: 0.5042 - accuracy: 0.7580 - val


Obtuvimos resultados similares para ambos embeddings.

A continuación utilizaremos un unit size mayor para la capa LSTM, para ver si obtenemos mejores resultados.

# Baseline + GloVe + mayor unit_size en LSTM

In [ ]:
model3 = build_baseline(lstm_size=128)
model3.summary()
model3.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test))
preds = model3.predict_classes(X_test)
metrics(preds, y_test)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 25, 200)           3080800   
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               168448    
_________________________________________________________________
dense_4 (Dense)              (None, 32)                4128      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 33        
Total params: 3,253,409
Trainable params: 172,609
Non-trainable params: 3,080,800
_________________________________________________________________
Epoch 1/5
191/191 [==============================] - 9s 47ms/step - loss: 0.4976 - accuracy: 0.7698 - va

El F1 Score apenas empeoró, y no hay grandes cambios en el precision/recall.
Ahora probaremos lo mismo pero con los otros embeddings.

# Baseline + w2v + mayor unit_size en LSTM

In [ ]:
model4 = build_baseline(lstm_size=128, emb_layer=emb_layer_w2v)
model4.summary()
model4.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test))
preds = model4.predict_classes(X_test)
metrics(preds, y_test)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 25, 300)           4621200   
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               219648    
_________________________________________________________________
dense_6 (Dense)              (None, 32)                4128      
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 33        
Total params: 4,845,009
Trainable params: 223,809
Non-trainable params: 4,621,200
_________________________________________________________________
Epoch 1/5
191/191 [==============================] - 11s 57ms/step - loss: 0.4978 - accuracy: 0.7657 - v

No vemos una mejora notoria en el F1 Score.

Perdimos precision pero ganamos un 6% en recall.

De momento dejaremos fijo este tamaño para LSTM y ahora probaremos con otra cantidad de neuronas en la capa densa.


In [ ]:
LSTM_SIZE = 128
HIDDEN_NEURONS = 512

# Baseline + GloVe Twitter + más hidden neurons

Intentaremos un salto agresivo, usaremos el cuadruple de neuronas ocultas y veremos qué pasa.

In [ ]:
model5 = build_baseline(lstm_size=LSTM_SIZE, hidden_neurons=HIDDEN_NEURONS)
model5.summary()
model5.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test))
preds = model5.predict_classes(X_test)
metrics(preds, y_test)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 25, 200)           3080800   
_________________________________________________________________
lstm_4 (LSTM)                (None, 128)               168448    
_________________________________________________________________
dense_8 (Dense)              (None, 512)               66048     
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 513       
Total params: 3,315,809
Trainable params: 235,009
Non-trainable params: 3,080,800
_________________________________________________________________
Epoch 1/5
191/191 [==============================] - 10s 51ms/step - loss: 0.4812 - accuracy: 0.7829 - v

# Baseline + w2v + más hidden neurons


In [ ]:
model6 = build_baseline(lstm_size=LSTM_SIZE, hidden_neurons=HIDDEN_NEURONS, emb_layer=emb_layer_w2v)
model6.summary()
model6.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test))
preds = model6.predict_classes(X_test)
metrics(preds, y_test)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 25, 300)           4621200   
_________________________________________________________________
lstm_5 (LSTM)                (None, 128)               219648    
_________________________________________________________________
dense_10 (Dense)             (None, 512)               66048     
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 513       
Total params: 4,907,409
Trainable params: 286,209
Non-trainable params: 4,621,200
_________________________________________________________________
Epoch 1/5
191/191 [==============================] - 11s 59ms/step - loss: 0.4845 - accuracy: 0.7831 - v

En ambos casos llegamos a 0.78 de F1 Score por lo que mejoró un poco el modelo. Vamos a probar agregar un poco de regularización por medio de la capa de Dropout con la que cuenta el modelo base.

In [ ]:
model7 = build_baseline(lstm_size=LSTM_SIZE, hidden_neurons=HIDDEN_NEURONS, dropout_rate=0.25)
model7.summary()
model7.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test))
preds = model7.predict_classes(X_test)
metrics(preds, y_test)

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 25, 200)           3080800   
_________________________________________________________________
lstm_6 (LSTM)                (None, 128)               168448    
_________________________________________________________________
dense_12 (Dense)             (None, 512)               66048     
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 513       
Total params: 3,315,809
Trainable params: 235,009
Non-trainable params: 3,080,800
_________________________________________________________________
Epoch 1/5
191/191 [==============================] - 10s 50ms/step - loss: 0.4963 - accuracy: 0.7785 - v

In [ ]:
model8 = build_baseline(lstm_size=LSTM_SIZE, hidden_neurons=HIDDEN_NEURONS, emb_layer=emb_layer_w2v, dropout_rate=0.25)
model8.summary()
model8.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test))
preds = model8.predict_classes(X_test)
metrics(preds, y_test)

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 25, 300)           4621200   
_________________________________________________________________
lstm_7 (LSTM)                (None, 128)               219648    
_________________________________________________________________
dense_14 (Dense)             (None, 512)               66048     
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 513       
Total params: 4,907,409
Trainable params: 286,209
Non-trainable params: 4,621,200
_________________________________________________________________
Epoch 1/5
191/191 [==============================] - 11s 58ms/step - loss: 0.4947 - accuracy: 0.7668 - v

En el caso de w2v sufrimos una pérdida de poco más de 1% en el F1 Score.

Mientras que en el caso de GloVe observamos se mantuvo casi igual el puntaje.

# Modelo de mayor complejidad

Ahora probaremos agregar algunas capas extra para ver cómo se comporta un modelo un poco más complejo basado en RNNs.

In [ ]:
# Esta arquitectura es un frankestein al que se llegó sacando ideas de tutoriales como el de TensorFlow donde
# recomiendan usar una capa Bidirectional para entrenar dos LSTMs, una que procesa los inputs en el orden que entran
# y otra que lo hace a la inversa. Se supone que así se puede sacar mayor contexto del texto.
# El SpatialDropout lo agregamos porque parece dar buenos resultados, así que no hay mucha más justificación que esa.

# El mejor submit fue con 0.2, 128, 512 y 0.25 en los sigueintes cuatro parámetros. Las dos capas densas tenían la misma cantidad de neuronas.
# Intento actual: modifiqué la cantidad de neuronas en la segunda capa densa.
# También agregamos recurrent_dropout para ver qué efecto produce en el resultado.
# Probaremos hacer un submit con este modelo y pasar a otras cosas.

SPATIAL_DROPOUT = 0.2
LTSM_SIZE = 128
HIDDEN_NEURONS = 512
DROPOUT = 0.5

m = Sequential()
m.add(emb_layer_w2v)
m.add(SpatialDropout1D(SPATIAL_DROPOUT))
m.add(Bidirectional(LSTM(LSTM_SIZE, return_sequences=True, recurrent_dropout=0.2)))
m.add(GlobalMaxPooling1D())
m.add(Dense(HIDDEN_NEURONS, activation='relu'))
m.add(Dropout(DROPOUT))
m.add(Dense(HIDDEN_NEURONS / 2, activation='relu'))
m.add(Dropout(DROPOUT))
m.add(Dense(1, activation='sigmoid'))
m.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
m.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 25, 300)           4621200   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 25, 300)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 25, 256)           439296    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 256)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 512)               131584    
_________________________________________________________________
dropout_10 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_20 (Dense)             (None, 256)              

In [ ]:
m.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test))

Epoch 1/5
191/191 [==============================] - 32s 168ms/step - loss: 0.5125 - accuracy: 0.7544 - val_loss: 0.4479 - val_accuracy: 0.8050
Epoch 2/5
191/191 [==============================] - 31s 164ms/step - loss: 0.4467 - accuracy: 0.8025 - val_loss: 0.4564 - val_accuracy: 0.7997
Epoch 3/5
191/191 [==============================] - 32s 169ms/step - loss: 0.4175 - accuracy: 0.8235 - val_loss: 0.4647 - val_accuracy: 0.7991
Epoch 4/5
191/191 [==============================] - 33s 172ms/step - loss: 0.3821 - accuracy: 0.8358 - val_loss: 0.4278 - val_accuracy: 0.8102
Epoch 5/5
191/191 [==============================] - 33s 172ms/step - loss: 0.3576 - accuracy: 0.8507 - val_loss: 0.4394 - val_accuracy: 0.8188


In [ ]:
preds = m.predict_classes(X_test)
metrics(preds, y_test)

Verdaderos Negativos: 746
Falsos Negativos: 168
Verdaderos Positivos: 501
Falsos Positivos: 108

precision score: 0.8226600985221675
recall score: 0.7488789237668162
f1 score: 0.7840375586854461


In [ ]:
m.fit(padded_vecs_train, train_target, epochs=epochs)
k_preds = m.predict_classes(padded_vecs_test)

results = df_test[['id']]
results['target'] = k_preds
results.shape

Epoch 1/5
238/238 [==============================] - 36s 151ms/step - loss: 0.1899 - accuracy: 0.9246
Epoch 2/5
238/238 [==============================] - 38s 162ms/step - loss: 0.1743 - accuracy: 0.9343
Epoch 3/5
238/238 [==============================] - 37s 154ms/step - loss: 0.1682 - accuracy: 0.9320
Epoch 4/5
238/238 [==============================] - 37s 155ms/step - loss: 0.1587 - accuracy: 0.9377
Epoch 5/5
238/238 [==============================] - 38s 158ms/step - loss: 0.1365 - accuracy: 0.9436


(3263, 2)

In [ ]:
results.to_csv('lstm-v2.csv', index=False)